In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import keras
from keras.models import load_model

# definte relu6
from tensorflow.python.keras import backend as K
def relu6(x):
    return K.relu(x, max_value=6)

# model = load_model('models/mobilenet_1.0_224_2018_06_18_16_08_08/face_attrib_mobilenet_1.0_224.10-0.18-0.17.hdf5', 
#                    custom_objects={'relu6': relu6})
# input_shape = (224, 224)
# model = load_model('models/mobilenet_1.0_192_2018_06_19_14_57_50/face_attrib_mobilenet_1.0_192.09-0.19-0.18.hdf5', 
#                    custom_objects={'relu6': relu6})
# input_shape = (192, 192)
# model = load_model('models/mobilenet_0.75_224_2018_06_20_11_40_04/face_attrib_mobilenet_0.75_224.11-0.19-0.17.hdf5', 
#                    custom_objects={'relu6': relu6})
# input_shape = (224, 224)
# model = load_model('models/mobilenet_0.5_224_2018_06_20_17_04_49/face_attrib_mobilenet_0.5_224.14-0.19-0.18.hdf5', 
#                    custom_objects={'relu6': relu6})
# input_shape = (224, 224)

# the following gives 45 outputs including ethnicities
# model = load_model('models/mobilenet_1.0_224_2018_06_13_17_27_59/face_attrib_mobilenet_1.0_224.25-0.32-0.08.hdf5', 
#                   custom_objects={'relu6': relu6})
model = load_model('models/mobilenet_0.5_224_2018_07_19_15_15_05/face_attrib_mobilenet_0.5_224.22-0.17-0.15.hdf5', 
                  custom_objects={'relu6': relu6})

input_shape = (224, 224)

Using TensorFlow backend.


In [2]:
import os
import numpy as np
from data import load_attributes
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
data_dir = '/data/celeba'
train_images_path = os.path.join(data_dir, 'Img/img_align_celeba_crop_middle_train/')
val_images_path = os.path.join(data_dir, 'Img/img_align_celeba_crop_middle_val/')
test_images_path = os.path.join(data_dir, 'Img/img_align_celeba_crop_middle_test/')
attributes_path = os.path.join('.', 'celeba_header_lines_45.p')
multilabel_dict, labels = load_attributes(attributes_path)
import cv2

def preprocess_image(image):
    image = image / 255.0
    image = image - 0.5
    image = image * 2.0
    return image

def get_data_in_X_Y(images_path, multilabel_dict):
    X = []
    Y = []
    for img_path in os.listdir(images_path):
        real_img_path = os.path.join(images_path, img_path)
        img = image.load_img(real_img_path, target_size=input_shape)
        x = image.img_to_array(img)
        # x = cv2.cvtColor(x, cv2.COLOR_RGB2BGR)
        x = preprocess_image(x)
        if img_path in multilabel_dict:
            X.append(x)
            Y.append(multilabel_dict[img_path])
    X=np.array(X)
    Y=np.array(Y)
    return X, Y

def eval_image_path(images_path, multilabel_dict, model, limit = 20000):
    img_paths = sorted(os.listdir(images_path))
    start = 0
    end = limit
    Y_predicted_all = []
    Y_gt_all = []
    images = []
    while start < len(img_paths):
        X = []
        Y = []
        for img_path in img_paths[start:end]:
            real_img_path = os.path.join(images_path, img_path)
            img = image.load_img(real_img_path, target_size=input_shape)
            x = image.img_to_array(img)
            # x = cv2.cvtColor(x, cv2.COLOR_RGB2BGR)
            x = preprocess_image(x)
            if img_path in multilabel_dict:
                X.append(x)
                Y.append(multilabel_dict[img_path])
                images.append(img_path)
        Y_predicted = model.predict(np.array(X), batch_size=64, verbose=1)
        Y_gt_all.extend(Y)
        Y_predicted_all.extend(Y_predicted)
        start = start + limit
        end = end + limit
    return Y_predicted_all, Y_gt_all, images

train_Y_predicted, train_Y_gt, train_img_paths = eval_image_path(train_images_path + 'all', multilabel_dict, model)
train_Y_predicted_rounded = np.round(train_Y_predicted)
val_Y_predicted, val_Y_gt, val_img_paths = eval_image_path(val_images_path + 'all', multilabel_dict, model)
val_Y_predicted_rounded = np.round(val_Y_predicted)
test_Y_predicted, test_Y_gt, test_img_paths = eval_image_path(test_images_path + 'all', multilabel_dict, model)
test_Y_predicted_rounded = np.round(test_Y_predicted)

19962/19962 [==============================] - 25s 1ms/step


In [10]:
def get_accuracies(predicted, gt):
    # assert predicted.shape == gt.shape, '{} != {}'.format(predicted.shape, gt.shape) 
    num_cols = len(predicted[0])
    num_rows = len(predicted)
    accuracies = []
    for i in range(num_cols):
        accuracies.append(0)
    for j in range(num_rows):
        for i in range(num_cols):
            if predicted[j][i] == gt[j][i]:
                accuracies[i] += 1
    for i in range(num_cols):
        accuracies[i] = accuracies[i] / 1.0 / num_rows
    return accuracies

val_accuracies = get_accuracies(val_Y_predicted_rounded, val_Y_gt)
print('-------------------- validation accuracies -------------------------')
print('mean: {:.04f}'.format(np.mean(val_accuracies)))
for i in range(len(labels)):
    print('{}, {:.04f}'.format(labels[i], val_accuracies[i]))
test_accuracies = get_accuracies(test_Y_predicted_rounded, test_Y_gt)
print('-------------------- test accuracies -------------------------')
print('mean: {:.04f}'.format(np.mean(test_accuracies)))
for i in range(len(labels)):
    print('{}, {:.04f}'.format(labels[i], test_accuracies[i]))
train_accuracies = get_accuracies(train_Y_predicted_rounded, train_Y_gt)
print('-------------------- train accuracies -------------------------')
print('mean: {:.04f}'.format(np.mean(train_accuracies)))
for i in range(len(labels)):
    print('{}, {:.04f}'.format(labels[i], train_accuracies[i]))    



-------------------- validation accuracies -------------------------
mean: 0.9244
asianindian, 0.9897
eastasian, 0.9801
african, 0.9876
latino, 0.9601
caucasian, 0.9382
5_o_Clock_Shadow, 0.9391
Arched_Eyebrows, 0.8622
Attractive, 0.8170
Bags_Under_Eyes, 0.8322
Bald, 0.9895
Bangs, 0.9612
Big_Lips, 0.8052
Big_Nose, 0.8329
Black_Hair, 0.9189
Blond_Hair, 0.9590
Blurry, 0.9683
Brown_Hair, 0.8633
Bushy_Eyebrows, 0.9265
Chubby, 0.9554
Double_Chin, 0.9669
Eyeglasses, 0.9960
Goatee, 0.9704
Gray_Hair, 0.9798
Heavy_Makeup, 0.9264
High_Cheekbones, 0.8913
Male, 0.9885
Mouth_Slightly_Open, 0.9417
Mustache, 0.9638
Narrow_Eyes, 0.9298
No_Beard, 0.9629
Oval_Face, 0.7618
Pale_Skin, 0.9668
Pointy_Nose, 0.7749
Receding_Hairline, 0.9460
Rosy_Cheeks, 0.9531
Sideburns, 0.9746
Smiling, 0.9369
Straight_Hair, 0.8511
Wavy_Hair, 0.8637
Wearing_Earrings, 0.9183
Wearing_Hat, 0.9913
Wearing_Lipstick, 0.9295
Wearing_Necklace, 0.8836
Wearing_Necktie, 0.9567
Young, 0.8842
-------------------- test accuracies ----------

In [13]:
race_predicted_dict = {}
race_gt_dict = {}

def extend_race_dict(img_paths, predicted, gt, race_predicted_dict, race_gt_dict):
    for ind in range(len(img_paths)):
        race_predicted_dict[img_paths[ind]] = predicted[ind]
        race_gt_dict[img_paths[ind]] = gt[ind]

extend_race_dict(train_img_paths, train_Y_predicted_rounded, train_Y_gt, race_predicted_dict, race_gt_dict)
extend_race_dict(test_img_paths, test_Y_predicted_rounded, test_Y_gt, race_predicted_dict, race_gt_dict)
extend_race_dict(val_img_paths, val_Y_predicted_rounded, val_Y_gt, race_predicted_dict, race_gt_dict)

import pickle
pickle.dump({'gt': race_gt_dict, 'predicted': race_predicted_dict}, open('race_diff.p', 'wb'))

In [16]:
new_images_path = os.path.join('/data/lfw/lfw_all_funneled_face_crop_l_0.3_r_0.3_t_0.4_d_0.2/all/')
new_attributes_path = os.path.join('.', 'lfw_header_lines_40.p')
new_multilabel_dict, new_labels = load_attributes(new_attributes_path)
new_X, new_Y_gt = get_data_in_X_Y(new_images_path, new_multilabel_dict)


In [17]:
new_Y_predicted = model.predict(new_X, batch_size=64, verbose=1)
new_Y_predicted_rounded = np.round(new_Y_predicted)

13143/13143 [==============================] - 22s 2ms/step


In [18]:
new_accuracies = get_accuracies(new_Y_predicted_rounded, new_Y_gt)
print('-------------------- test accuracies -------------------------')
print('mean: {:.04f}'.format(np.mean(new_accuracies)))
for i in range(len(labels)):
    print('{}, {:.04f}'.format(labels[i], new_accuracies[i]))

-------------------- test accuracies -------------------------
mean: 0.7384
5_o_Clock_Shadow, 0.6203
Arched_Eyebrows, 0.7698
Attractive, 0.6747
Bags_Under_Eyes, 0.6659
Bald, 0.9085
Bangs, 0.8598
Big_Lips, 0.6674
Big_Nose, 0.5404
Black_Hair, 0.8745
Blond_Hair, 0.9610
Blurry, 0.8304
Brown_Hair, 0.6669
Bushy_Eyebrows, 0.5291
Chubby, 0.6743
Double_Chin, 0.6810
Eyeglasses, 0.9216
Goatee, 0.7778
Gray_Hair, 0.8535
Heavy_Makeup, 0.9340
High_Cheekbones, 0.8401
Male, 0.9781
Mouth_Slightly_Open, 0.7860
Mustache, 0.9101
Narrow_Eyes, 0.4376
No_Beard, 0.7962
Oval_Face, 0.4810
Pale_Skin, 0.5002
Pointy_Nose, 0.3336
Receding_Hairline, 0.5225
Rosy_Cheeks, 0.8064
Sideburns, 0.7043
Smiling, 0.8949
Straight_Hair, 0.5862
Wavy_Hair, 0.5825
Wearing_Earrings, 0.8934
Wearing_Hat, 0.8974
Wearing_Lipstick, 0.9301
Wearing_Necklace, 0.8057
Wearing_Necktie, 0.6875
Young, 0.7526


In [4]:
race_labels = ['asianindian', 'eastasian', 'african', 'latino', 'caucasian']

race_dict = {}
def extract_race_dict(predicted_rounded, predicted, img_paths, race_dict):
    for i in range(len(predicted)):
        one_of_races = False
        for j in range(5):
            if predicted_rounded[i][j+1] == 1:
                one_of_races = True
                if race_labels[j] not in race_dict:
                    race_dict[race_labels[j]] = []
                race_dict[race_labels[j]].append(img_paths[i])
        if one_of_races == False:
            j = np.argmax(predicted[i][1:6])
            if race_labels[j] not in race_dict:
                race_dict[race_labels[j]] = []
            race_dict[race_labels[j]].append(img_paths[i])

extract_race_dict(train_Y_predicted_rounded, train_Y_predicted, train_img_paths, race_dict)
extract_race_dict(val_Y_predicted_rounded, val_Y_predicted, val_img_paths, race_dict)
extract_race_dict(test_Y_predicted_rounded, test_Y_predicted, test_img_paths, race_dict)

In [5]:
import pickle
pickle.dump(race_dict, open('race_dict.p', 'wb'))

In [8]:
val_Y_predicted_rounded.shape

(19867, 45)